In [1]:
import numpy as np
import pandas as pd
from random import random
from random import seed
from math import exp
from sklearn.datasets import make_classification
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
#SAMPLE DATA POINTS
sampleNum = 200

# NO OF FEATURES 
featureNum = 4

#REDUNDENT NUMBER
redundantNum = 1

#CLASSES NUMBER
classesNum = 2

In [6]:
#DATASET READING
X, Y = make_classification(n_samples=sampleNum, n_features=featureNum, n_redundant=redundantNum, n_classes=classesNum)
df = pd.DataFrame(X, columns=['feature1', 'feature2', 'feature3', 'feature4'])
df['label'] = Y
df.to_csv("dataset1.csv")
df=pd.read_csv('dataset1.csv',index_col=0)


In [7]:
#FUNCTION REQUIRED FOR TRANSFER
def transferfuntion(activater):
    return 1.0 / (1.0 + exp(-activater))

In [8]:
#TRANSFERING DEVRIVATIVE FUNTION
def transferDerivativefuntion(input):
    return input * (1.0 - input)


In [9]:
#FUNCTION FOR PREDICTIO
def predictingfunction(netw, row):
    outpt = forwardPropagatefunction(netw, row)
    return outpt.index(max(outpt))

In [10]:
#ACTIVATION FUNCTION
def activatefunction(wtd, input):
    activator=wtd[-1]
    for i in range(len(wtd)-1):
        activator+=wtd[i]*input[i]
    return activator

In [11]:
# INITIALIZING netwORK
def networkInitializingfunction(inputNum, hiddenNum, outputNum):
    netw=list()
    hiddenLayer = [{'weights':[random() for i in range(inputNum + 1)]} for i in range(hiddenNum)]
    netw.append(hiddenLayer)
    outputLayer = [{'weights':[random() for i in range(hiddenNum + 1)]} for i in range(outputNum)]
    netw.append(outputLayer)
    return netw

In [12]:
#BACKWARD PROPOGATION FUNCTION
def backwardPropagatefunction(netw, excpted):
    for i in reversed(range(len(netw))):
        layr = netw[i]
        err = list()
        if i != len(netw)-1:
            for j in range(len(layr)):
                error = 0.0
                for neur in netw[i + 1]:
                    error += (neur['weights'][j] * neur['delta'])
                err.append(error)
        else:
            for j in range(len(layr)):
                neur = layr[j]
                err.append(excpted[j] - neur['output'])
        for j in range(len(layr)):
            neur = layr[j]
            neur['delta'] = err[j] * transferDerivativefuntion(neur['output'])

In [13]:
#FORWARD PORPOGATION
def forwardPropagatefunction(netw,data):
    rwIp=data
    for i in netw:
        newRw=[]
        for j in i:
            activater=activatefunction(j['weights'], rwIp)
            j['output']=transferfuntion(activater)
            newRw.append(j['output'])
        rwIp=newRw
    return rwIp

In [14]:
#netwORK TRAINING FUNCTION
def trainnetworkfuntion(netw, training, l_rate, epochNum, outputNum):
    for epoch in range(epochNum):
        sum_err = 0
        for row in training:
            outpt = forwardPropagatefunction(netw, row)
            excpted = [0 for i in range(outputNum)]
            excpted[int(row[-1])] = 1
            sum_err += sum([(excpted[i]-outpt[i])**2 for i in range(len(excpted))])
            backwardPropagatefunction(netw, excpted)
            updatingWeightsfuntion(netw, row, l_rate)
        print('Loop=%d, learn_rate=%.3f, Error=%.3f' % (epoch, l_rate, sum_err))


In [15]:
#WEIGHTS UPDATION FUNCTION
def updatingWeightsfuntion(netw, row, l_rate):
    for i in range(len(netw)):
        inp=row[:-1]
        if i!=0:
            inp=[neur['output'] for neur in netw[i-1]]
        for neur in netw[i]:
            for j in range(len(inp)):
                neur['weights'][j]+=l_rate*neur['delta']*inp[j]
            neur['weights'][-1]+=l_rate*neur['delta']

In [16]:
#ARRAY IN DATASET
dataset=np.array(df[:])
dataset

array([[ 2.84806869e-01,  8.72383573e-01, -1.23084784e+00,
         7.54366056e-01,  1.00000000e+00],
       [-8.21708093e-01, -2.17883238e+00,  9.83342523e-01,
         1.53811394e+00,  0.00000000e+00],
       [-3.77809570e-02, -2.00870799e+00,  3.08108626e+00,
        -2.14125028e+00,  0.00000000e+00],
       [-3.88759601e-01,  5.86548954e-01, -1.30543383e+00,
         1.28937990e+00,  1.00000000e+00],
       [-1.21777594e-02, -1.76636551e+00,  2.92252170e+00,
        -2.23181158e+00,  0.00000000e+00],
       [-4.73265952e-01,  8.81880868e-01,  2.09195786e-01,
        -1.61642237e+00,  1.00000000e+00],
       [ 1.69103702e+00,  5.03355254e-01, -9.51167095e-01,
         8.29700800e-01,  1.00000000e+00],
       [ 5.12190811e-01,  8.46220159e-01, -1.35896240e-02,
        -1.20024920e+00,  1.00000000e+00],
       [-3.69756434e-02,  1.74241133e+00, -1.92715283e+00,
         6.37193950e-01,  1.00000000e+00],
       [-5.15323669e-01,  1.01697753e+00, -9.95258867e-01,
         1.59865454e-01

In [17]:
#SETTING INPUT AND OUTPUT
inputNum = len(dataset[0]) - 1
outputNum = len(set([row[-1] for row in dataset]))
print(inputNum,outputNum)

4 2


In [18]:
#SPLIT DATASET
trainDatasetVar=dataset[:150]
testDatasetVar=dataset[150:]

In [19]:
#DATASET INTO netw
netw=networkInitializingfunction(inputNum,1,outputNum)
trainnetworkfuntion(netw, trainDatasetVar, 0.5, 100, outputNum)


Loop=0, learn_rate=0.500, Error=69.753
Loop=1, learn_rate=0.500, Error=43.982
Loop=2, learn_rate=0.500, Error=33.421
Loop=3, learn_rate=0.500, Error=29.605
Loop=4, learn_rate=0.500, Error=27.907
Loop=5, learn_rate=0.500, Error=27.017
Loop=6, learn_rate=0.500, Error=26.494
Loop=7, learn_rate=0.500, Error=26.159
Loop=8, learn_rate=0.500, Error=25.931
Loop=9, learn_rate=0.500, Error=25.768
Loop=10, learn_rate=0.500, Error=25.646
Loop=11, learn_rate=0.500, Error=25.554
Loop=12, learn_rate=0.500, Error=25.480
Loop=13, learn_rate=0.500, Error=25.422
Loop=14, learn_rate=0.500, Error=25.374
Loop=15, learn_rate=0.500, Error=25.334
Loop=16, learn_rate=0.500, Error=25.300
Loop=17, learn_rate=0.500, Error=25.271
Loop=18, learn_rate=0.500, Error=25.246
Loop=19, learn_rate=0.500, Error=25.225
Loop=20, learn_rate=0.500, Error=25.206
Loop=21, learn_rate=0.500, Error=25.189
Loop=22, learn_rate=0.500, Error=25.174
Loop=23, learn_rate=0.500, Error=25.161
Loop=24, learn_rate=0.500, Error=25.149
Loop=25, l

In [20]:
#WEIGHTS OF netwORK
for layr in netw:
    print(layr)

[{'weights': [-0.4371076774887024, 4.760731086837036, -2.5733402778980135, -1.1937893911676414, -0.4358667673825133], 'output': 0.9879573860661551, 'delta': 0.0019714516958389016}]
[{'weights': [-5.354183061650958, 3.4061890930724035], 'output': 0.13374957986938046, 'delta': -0.015496310553018693}, {'weights': [5.354297357742987, -3.406280492259578], 'output': 0.8662529760503059, 'delta': 0.015495764018721022}]


In [21]:
#TESTING DATASET
testSet=[]
pred=[]
for row in testDatasetVar:
    prediction = predictingfunction(netw, row)
    testSet.append(row[-1])
    pred.append(prediction)
print()
print("TEST DATASET")
print("Confusion Matrix is: ",confusion_matrix(testSet,pred))
print("Accuracy is: ",accuracy_score(testSet,pred))
print("Precision is: ",precision_score(testSet, pred))
print("recall is: ",recall_score(testSet, pred))


TEST DATASET
Confusion Matrix is:  [[20  3]
 [ 3 24]]
Accuracy is:  0.88
Precision is:  0.8888888888888888
recall is:  0.8888888888888888


In [22]:
#TRAINING DATASET
trainSet=[]
pred=[]
for row in trainDatasetVar:
    prediction = predictingfunction(netw, row)
    trainSet.append(int(row[-1]))
    pred.append(prediction)

In [23]:
print()
print("TRAIN DATASET")
print("Confusion Matrix is: ",confusion_matrix(trainSet,pred))
print("Accuracy is: ",accuracy_score(trainSet,pred))
print("Precision is: ",precision_score(trainSet, pred))
print("recall is: ",recall_score(trainSet, pred))


TRAIN DATASET
Confusion Matrix is:  [[68 11]
 [ 3 68]]
Accuracy is:  0.9066666666666666
Precision is:  0.8607594936708861
recall is:  0.9577464788732394
